In [2]:
from pyspark.sql import SparkSession, Window
from pyspark.sql.functions import *

In [3]:
spark = SparkSession \
        .builder \
        .master("local")\
        .appName("USVehicleAccidentAnalysis") \
        .getOrCreate()

21/07/11 16:23:01 WARN Utils: Your hostname, AS-MAC-0048.local resolves to a loopback address: 127.0.0.1; using 192.168.186.161 instead (on interface en0)
21/07/11 16:23:01 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/07/11 16:23:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [294]:
df.printSchema()

root
 |-- DRVR_ZIP: string (nullable = true)
 |-- count: long (nullable = false)



In [291]:
spark.stop()

In [119]:
df_charges.printSchema()

root
 |-- CRASH_ID: string (nullable = true)
 |-- UNIT_NBR: string (nullable = true)
 |-- PRSN_NBR: string (nullable = true)
 |-- CHARGE: string (nullable = true)
 |-- CITATION_NBR: string (nullable = true)



In [5]:
df_charges = spark.read.csv("Data/Charges_use.csv", header=True)
df_damages = spark.read.csv("Data/Damages_use.csv", header=True)
df_endorse = spark.read.csv("Data/Endorse_use.csv", header=True)
df_pp = spark.read.csv("Data/Primary_Person_use.csv", header=True)
df_Units = spark.read.csv("Data/Units_use.csv", header=True)
df_restrict = spark.read.csv("Data/Restrict_use.csv", header=True)

In [23]:
df_Units.select("VEH_COLOR_ID").distinct().show(100, truncate = False)

+------------+
|VEH_COLOR_ID|
+------------+
|RED         |
|TEA         |
|BLK         |
|ONG         |
|YEL         |
|GLD         |
|NA          |
|98          |
|99          |
|CAM         |
|BRZ         |
|TRQ         |
|SIL         |
|CPR         |
|BRO         |
|PLE         |
|BLU         |
|MAR         |
|BGE         |
|PNK         |
|WHI         |
|TAN         |
|MUL         |
|GRN         |
|GRY         |
+------------+



In [37]:
df_Units.groupby("VEH_MAKE_ID").count().orderBy(col("count").desc()).show()

+---------------+-----+
|     VEH_MOD_ID|count|
+---------------+-----+
|             NA|14039|
|        UNKNOWN|10348|
|           F150| 8612|
|      SILVERADO| 4665|
|         ACCORD| 4156|
|          CAMRY| 4073|
|       RAM 1500| 3983|
|         ALTIMA| 3258|
|        COROLLA| 3168|
|          CIVIC| 3104|
|          TAHOE| 2438|
|        MUSTANG| 2426|
|       EXPLORER| 2350|
|          C1500| 2011|
|           F250| 1967|
|         MALIBU| 1957|
|         IMPALA| 1894|
|          FOCUS| 1882|
|SILVERADO C1500| 1873|
|     EXPEDITION| 1713|
+---------------+-----+
only showing top 20 rows



In [27]:
df_Units.columns

['CRASH_ID',
 'UNIT_NBR',
 'UNIT_DESC_ID',
 'VEH_PARKED_FL',
 'VEH_HNR_FL',
 'VEH_LIC_STATE_ID',
 'VIN',
 'VEH_MOD_YEAR',
 'VEH_COLOR_ID',
 'VEH_MAKE_ID',
 'VEH_MOD_ID',
 'VEH_BODY_STYL_ID',
 'EMER_RESPNDR_FL',
 'OWNR_ZIP',
 'FIN_RESP_PROOF_ID',
 'FIN_RESP_TYPE_ID',
 'VEH_DMAG_AREA_1_ID',
 'VEH_DMAG_SCL_1_ID',
 'FORCE_DIR_1_ID',
 'VEH_DMAG_AREA_2_ID',
 'VEH_DMAG_SCL_2_ID',
 'FORCE_DIR_2_ID',
 'VEH_INVENTORIED_FL',
 'VEH_TRANSP_NAME',
 'VEH_TRANSP_DEST',
 'CONTRIB_FACTR_1_ID',
 'CONTRIB_FACTR_2_ID',
 'CONTRIB_FACTR_P1_ID',
 'VEH_TRVL_DIR_ID',
 'FIRST_HARM_EVT_INV_ID',
 'INCAP_INJRY_CNT',
 'NONINCAP_INJRY_CNT',
 'POSS_INJRY_CNT',
 'NON_INJRY_CNT',
 'UNKN_INJRY_CNT',
 'TOT_INJRY_CNT',
 'DEATH_CNT']

In [55]:
# df_Units.groupby("VEH_LIC_STATE_ID").count().orderBy(col("count").desc()).show(100)
df_Units.filter(col("VEH_LIC_STATE_ID").cast("int").isNull()).groupby("VEH_LIC_STATE_ID").count().orderBy(col("count").desc()).show(100)

+----------------+------+
|VEH_LIC_STATE_ID| count|
+----------------+------+
|              TX|156997|
|              NA|  6065|
|              UN|  1791|
|              OK|   970|
|              LA|   782|
|              NM|   731|
|              IN|   544|
|              MX|   493|
|              CA|   482|
|              FL|   416|
|              IL|   370|
|              AR|   350|
|              TN|   304|
|              MS|   244|
|              AZ|   236|
|              KS|   186|
|              MO|   177|
|              GA|   177|
|              CO|   175|
|              NC|   159|
|              AL|   145|
|              OH|   127|
|              MI|   122|
|              MN|   113|
|              WI|   101|
|              VA|    98|
|              NE|    88|
|              WA|    85|
|              IA|    79|
|              UT|    75|
|              OR|    71|
|              PA|    70|
|              SC|    63|
|              CD|    57|
|              NY|    50|
|           

In [22]:
df_pp.filter(df_pp.DRVR_LIC_TYPE_ID.isin(["DRIVER LICENSE", "COMMERCIAL DRIVER LIC."])).select("DRVR_LIC_TYPE_ID").distinct().show(100, truncate = False)

+----------------------+
|DRVR_LIC_TYPE_ID      |
+----------------------+
|COMMERCIAL DRIVER LIC.|
|DRIVER LICENSE        |
+----------------------+



In [13]:
df_charges.select("CHARGE").filter(df_charges.CHARGE.contains("SPEED")).distinct().show(100, truncate = False)

+--------------------------------------------------------------+
|CHARGE                                                        |
+--------------------------------------------------------------+
|UNSAFE SPEED/ DISPLAY EX REG                                  |
|FAILURE TO CONTROL SPEED, NO DRIVER LICENSE                   |
|FAILURE TO CONTROL SPEED ACCIDENT                             |
|FT CONTROL SPEED C/A                                          |
|TC545.351 FAIL TO CONTROL SPEED                               |
|UNSAFE SPEED -CONDITIONS/RAIN                                 |
|FAIL TO CONTROL SPEED/ACCIDENT                                |
|FAIL TO CONTROL SPEED (RUST)                                  |
|FAILT TO CONTROL MOTOR VEHICLE -SPEED                         |
|FAILURE TO CONTROL SPEED-ACC, NO DL                           |
|FAIL TO CONTROL SPEED- ACC                                    |
|COLLISION BY DRIVING UNSAFE SPEED                             |
|UNSAFE SPEED  TXTRC 545.

In [43]:
df = df_charges.join(df_pp, on=['CRASH_ID'], how='inner').\
join(df_Units,  on=['CRASH_ID'], how='inner').\
filter(df_charges.CHARGE.contains("SPEED")).\
filter(df_pp.DRVR_LIC_TYPE_ID.isin(["DRIVER LICENSE", "COMMERCIAL DRIVER LIC."])).\
groupby("VEH_COLOR_ID", "VEH_LIC_STATE_ID", "VEH_MAKE_ID").count().orderBy(col("count").desc())

df_Units.filter(col("VEH_LIC_STATE_ID").cast("int").isNull()).groupby("VEH_LIC_STATE_ID").count().orderBy(col("count").desc()).show(100)

In [56]:
top_25_state_list = [row[0] for row in df_Units.filter(col("VEH_LIC_STATE_ID").cast("int").isNull()).groupby("VEH_LIC_STATE_ID").count().orderBy(col("count").desc()).limit(25).collect()]


In [60]:
top_10_used_vehicle_colors = [row[0] for row in df_Units.filter(df_Units.VEH_COLOR_ID != "NA").groupby("VEH_COLOR_ID").count().orderBy(col("count").desc()).limit(10).collect()]


In [61]:
top_10_used_vehicle_colors

['WHI', 'BLK', 'SIL', 'GRY', 'BLU', 'RED', 'GRN', 'MAR', 'TAN', 'GLD']

In [57]:
top_25_state_list

['TX',
 'NA',
 'UN',
 'OK',
 'LA',
 'NM',
 'IN',
 'MX',
 'CA',
 'FL',
 'IL',
 'AR',
 'TN',
 'MS',
 'AZ',
 'KS',
 'GA',
 'MO',
 'CO',
 'NC',
 'AL',
 'OH',
 'MI',
 'MN',
 'WI']

In [65]:
df_charges.join(df_pp, on=['CRASH_ID'], how='inner'). \
join(df_Units, on=['CRASH_ID'], how='inner'). \
filter(df_charges.CHARGE.contains("SPEED")). \
filter(df_pp.DRVR_LIC_TYPE_ID.isin(["DRIVER LICENSE", "COMMERCIAL DRIVER LIC."])).\
filter(df_Units.VEH_COLOR_ID.isin(top_10_used_vehicle_colors)).\
filter(df_Units.VEH_LIC_STATE_ID.isin(top_25_state_list)).\
groupby("VEH_MAKE_ID").count().\
orderBy(col("count").desc()).show()

+-------------+-----+
|  VEH_MAKE_ID|count|
+-------------+-----+
|         FORD|19205|
|    CHEVROLET|16860|
|       TOYOTA|11822|
|        DODGE| 7935|
|       NISSAN| 7332|
|        HONDA| 7189|
|          GMC| 3147|
|         JEEP| 2937|
|      HYUNDAI| 2678|
|          KIA| 2111|
| FREIGHTLINER| 1825|
|     CHRYSLER| 1692|
|        MAZDA| 1669|
|        LEXUS| 1548|
|   VOLKSWAGEN| 1407|
|          BMW| 1132|
|MERCEDES-BENZ| 1128|
|      PONTIAC| 1096|
|        BUICK| 1080|
|     CADILLAC| 1007|
+-------------+-----+
only showing top 20 rows



In [44]:
df.show(truncate = False)

+------------+----------------+-----------+-----+
|VEH_COLOR_ID|VEH_LIC_STATE_ID|VEH_MAKE_ID|count|
+------------+----------------+-----------+-----+
|WHI         |TX              |FORD       |5581 |
|WHI         |TX              |CHEVROLET  |4514 |
|NA          |NA              |NA         |4148 |
|BLK         |TX              |FORD       |3062 |
|BLK         |TX              |CHEVROLET  |2861 |
|WHI         |TX              |TOYOTA     |2480 |
|RED         |TX              |FORD       |2095 |
|SIL         |TX              |TOYOTA     |2079 |
|SIL         |TX              |CHEVROLET  |1888 |
|WHI         |TX              |DODGE      |1860 |
|GRY         |TX              |FORD       |1801 |
|GRY         |TX              |TOYOTA     |1750 |
|SIL         |TX              |FORD       |1744 |
|BLU         |TX              |FORD       |1712 |
|BLK         |TX              |TOYOTA     |1632 |
|GRY         |TX              |CHEVROLET  |1609 |
|BLK         |TX              |NISSAN     |1518 |


In [ ]:
.groupby("VEH_LIC_STATE_ID").count().orderBy(col("count").desc())

In [19]:
df_charges.show(truncate=False)

+--------+--------+--------+----------------------------------------+------------+
|CRASH_ID|UNIT_NBR|PRSN_NBR|CHARGE                                  |CITATION_NBR|
+--------+--------+--------+----------------------------------------+------------+
|14768622|1       |1       |DRIVING WHILE INTOXICATED               |null        |
|14838637|1       |1       |DWI                                     |1600000015  |
|14838641|1       |1       |RAN RED LIGHT SOLID (TC 544.007)        |L20440      |
|14838641|2       |1       |NO DRIVER'S LICENSE (TC521.025)         |L23141      |
|14838668|1       |1       |DRIVING WHILE INTOXICATED               |TX4IC50SRJD3|
|14838669|2       |1       |DWI W/BAC >.015                         |2015-000006 |
|14838670|1       |1       |DRIVING WHILE INTOXICATED               |2016-000003 |
|14838685|1       |1       |FAILED TO DRIVE SINGLE LANE, NOL, NO INS|138434825   |
|14838693|1       |1       |DRIVING WHILE INTOXICATED               |TX4IC60UKQND|
|148

In [8]:
df_damages.count()

24950

In [18]:
df_damages.show()

+--------+--------------------+
|CRASH_ID|    DAMAGED_PROPERTY|
+--------+--------------------+
|14768622|             MAILBOX|
|14768622|         YARD, GRASS|
|14838668|           GUARDRAIL|
|14838685|           ROAD SIGN|
|14838693|        2009 MAZDA 3|
|14838834|    CHAIN LINK FENCE|
|14838841|WOODED POLE ON SO...|
|14838842|CITY SIGN FOR TUR...|
|14838877|    FENCE-CHAIN LINK|
|14838977|LANDSCAPING AND M...|
|14839047|  APARTMENT BUILDING|
|14839047|         STREET SIGN|
|14839048|               HOUSE|
|14839314|          LIGHT POLE|
|14839330|MINOR DAMAGE TO W...|
|14839442|CITY OF SAN ANTON...|
|14839472|         METAL POLES|
|14839517|    WATER ATTENUATOR|
|14839519|        UTILITY POST|
|14839551|30 FEET OF GUARDRAIL|
+--------+--------------------+
only showing top 20 rows



In [9]:
df_endorse.count()

159818

In [17]:
df_endorse.show()

+--------+--------+------------------+
|CRASH_ID|UNIT_NBR|DRVR_LIC_ENDORS_ID|
+--------+--------+------------------+
|14768622|       1|              NONE|
|14838637|       1|              NONE|
|14838637|       2|              NONE|
|14838641|       1|              NONE|
|14838641|       2|        UNLICENSED|
|14838668|       1|              NONE|
|14838669|       1|              NONE|
|14838669|       2|              NONE|
|14838670|       1|              NONE|
|14838685|       1|        UNLICENSED|
|14838693|       1|              NONE|
|14838768|       1|OTHER/OUT OF STATE|
|14838768|       2|              NONE|
|14838834|       1|           UNKNOWN|
|14838841|       1|              NONE|
|14838842|       1|              NONE|
|14838842|       2|              NONE|
|14838866|       1|              NONE|
|14838877|       1|              NONE|
|14838972|       1|              NONE|
+--------+--------+------------------+
only showing top 20 rows



In [105]:
df_pp.select("PRSN_GNDR_ID", "DRVR_LIC_STATE_ID").filter(df_pp.PRSN_GNDR_ID == "FEMALE").groupby("DRVR_LIC_STATE_ID").count().orderBy(col("count").desc()).first().DRVR_LIC_STATE_ID


'Texas'

In [10]:
df_pp.count()

156954

In [51]:
df_pp.select("DRVR_ZIP").distinct().show(truncate=False)

+---------+
|DRVR_ZIP |
+---------+
|75007    |
|78073    |
|77371    |
|79849    |
|77339    |
|77303    |
|78382    |
|75602    |
|72472    |
|770598   |
|84606    |
|71047    |
|39645    |
|08648    |
|73463    |
|32812    |
|36526    |
|76686    |
|88277    |
|783720000|
+---------+
only showing top 20 rows



In [174]:
df_pp.select("CRASH_ID", "PRSN_ETHNICITY_ID").distinct().show()

+--------+-----------------+
|CRASH_ID|PRSN_ETHNICITY_ID|
+--------+-----------------+
|14842987|            BLACK|
|14845235|            WHITE|
|14847796|            WHITE|
|14848985|         HISPANIC|
|14849674|            WHITE|
|14849994|            ASIAN|
|14850477|            WHITE|
|14850795|            BLACK|
|14850895|            WHITE|
|14851253|            BLACK|
|14851478|         HISPANIC|
|14851529|            BLACK|
|14852255|         HISPANIC|
|14852370|            WHITE|
|14852671|            BLACK|
|14852682|            WHITE|
|14853923|            BLACK|
|14855453|            ASIAN|
|14856417|            WHITE|
|14858416|            BLACK|
+--------+-----------------+
only showing top 20 rows



In [215]:
w2 = Window.partitionBy("VEH_BODY_STYL_ID").orderBy(col("count").desc())

In [227]:
df_Units.filter(col("CONTRIB_FACTR_2_ID").contains("ALCOHOL")).select("CONTRIB_FACTR_2_ID").show()

+--------------------+
|  CONTRIB_FACTR_2_ID|
+--------------------+
|UNDER INFLUENCE -...|
|UNDER INFLUENCE -...|
|UNDER INFLUENCE -...|
|UNDER INFLUENCE -...|
|UNDER INFLUENCE -...|
|UNDER INFLUENCE -...|
|UNDER INFLUENCE -...|
|UNDER INFLUENCE -...|
|UNDER INFLUENCE -...|
|UNDER INFLUENCE -...|
|UNDER INFLUENCE -...|
|UNDER INFLUENCE -...|
|UNDER INFLUENCE -...|
|UNDER INFLUENCE -...|
|UNDER INFLUENCE -...|
|UNDER INFLUENCE -...|
|UNDER INFLUENCE -...|
|UNDER INFLUENCE -...|
|UNDER INFLUENCE -...|
|UNDER INFLUENCE -...|
+--------------------+
only showing top 20 rows



In [234]:
df = df_Units.join(df_pp, on=['CRASH_ID'], how='inner').\
dropna(subset=["DRVR_ZIP"]).\
filter(col("CONTRIB_FACTR_1_ID").contains("ALCOHOL") | col("CONTRIB_FACTR_2_ID").contains("ALCOHOL")).\
groupby("DRVR_ZIP").count().orderBy(col("count").desc()).limit(5)


In [235]:
df.printSchema()

root
 |-- DRVR_ZIP: string (nullable = true)
 |-- count: long (nullable = false)



In [220]:
df_Units.join(df_pp, on=['CRASH_ID'], how='inner').\
filter(~df_Units.VEH_BODY_STYL_ID.isin(["NA", "UNKNOWN", "NOT REPORTED", "OTHER  (EXPLAIN IN NARRATIVE)"])).\
filter(~df_pp.PRSN_ETHNICITY_ID.isin(["NA", "UNKNOWN"])).\
groupby("VEH_BODY_STYL_ID", "PRSN_ETHNICITY_ID").count().\
withColdf_Units.join(df_pp, on=['CRASH_ID'], how='inner').\
filter(~df_Units.VEH_BODY_STYL_ID.isin(["NA", "UNKNOWN"])).\
filter(~df_pp.PRSN_ETHNICITY_ID.isin(["NA", "UNKNOWN"])).\
groupby("VEH_BODY_STYL_ID", "PRSN_ETHNICITY_ID").count().\
withColumn("row", row_number().over(w2)).filter(col("row") == 1).drop("row").show()
umn("row", row_number().over(w2)).filter(col("row") == 1).drop("row").show(truncate=False)


+---------------------------------+-----------------+-----+
|VEH_BODY_STYL_ID                 |PRSN_ETHNICITY_ID|count|
+---------------------------------+-----------------+-----+
|BUS                              |HISPANIC         |391  |
|VAN                              |WHITE            |5291 |
|PICKUP                           |WHITE            |38609|
|SPORT UTILITY VEHICLE            |WHITE            |33902|
|PASSENGER CAR, 4-DOOR            |WHITE            |58312|
|FIRE TRUCK                       |WHITE            |112  |
|TRUCK                            |WHITE            |4204 |
|AMBULANCE                        |WHITE            |97   |
|POLICE CAR/TRUCK                 |WHITE            |366  |
|MOTORCYCLE                       |WHITE            |848  |
|YELLOW SCHOOL BUS                |WHITE            |264  |
|POLICE MOTORCYCLE                |HISPANIC         |3    |
|PASSENGER CAR, 2-DOOR            |WHITE            |9877 |
|TRUCK TRACTOR                    |WHITE

In [173]:
df_Units.select("VEH_BODY_STYL_ID").distinct().show()

+--------------------+
|    VEH_BODY_STYL_ID|
+--------------------+
|                 BUS|
|                  NA|
|                 VAN|
|              PICKUP|
|SPORT UTILITY VEH...|
|PASSENGER CAR, 4-...|
|          FIRE TRUCK|
|               TRUCK|
|             UNKNOWN|
|           AMBULANCE|
|    POLICE CAR/TRUCK|
|          MOTORCYCLE|
|   YELLOW SCHOOL BUS|
|   POLICE MOTORCYCLE|
|PASSENGER CAR, 2-...|
|       TRUCK TRACTOR|
|      FARM EQUIPMENT|
|NEV-NEIGHBORHOOD ...|
|OTHER  (EXPLAIN I...|
|        NOT REPORTED|
+--------------------+



In [16]:
df_Units.head(2)

[Row(CRASH_ID='14768622', UNIT_NBR='1', UNIT_DESC_ID='MOTOR VEHICLE', VEH_PARKED_FL='N', VEH_HNR_FL='N', VEH_LIC_STATE_ID='TX', VIN='4S2CK57D1X4381118', VEH_MOD_YEAR='1999', VEH_COLOR_ID='GRY', VEH_MAKE_ID='ISUZU', VEH_MOD_ID='AMIGO', VEH_BODY_STYL_ID='PASSENGER CAR, 4-DOOR', EMER_RESPNDR_FL='Y', OWNR_ZIP='77357', FIN_RESP_PROOF_ID='2', FIN_RESP_TYPE_ID='NA', VEH_DMAG_AREA_1_ID='FRONT END DAMAGE CONCENTRATED IMPACT', VEH_DMAG_SCL_1_ID='DAMAGED 3', FORCE_DIR_1_ID='1', VEH_DMAG_AREA_2_ID='LEFT SIDE AND TOP DAMAGE ROLLOVER EFFECTS', VEH_DMAG_SCL_2_ID='DAMAGED 4', FORCE_DIR_2_ID='3', VEH_INVENTORIED_FL='Y', VEH_TRANSP_NAME='VALENTIN TOWING', VEH_TRANSP_DEST='20288 FM 1314 PORTER, TX', CONTRIB_FACTR_1_ID='UNDER INFLUENCE - ALCOHOL', CONTRIB_FACTR_2_ID='NA', CONTRIB_FACTR_P1_ID='NA', VEH_TRVL_DIR_ID='EAST', FIRST_HARM_EVT_INV_ID='Y', INCAP_INJRY_CNT='0', NONINCAP_INJRY_CNT='0', POSS_INJRY_CNT='0', NON_INJRY_CNT='1', UNKN_INJRY_CNT='0', TOT_INJRY_CNT='0', DEATH_CNT='0'),
 Row(CRASH_ID='148386

In [223]:
df_Units.select("CONTRIB_FACTR_P1_ID").distinct().show(truncate=False)

+----------------------------------------+
|CONTRIB_FACTR_P1_ID                     |
+----------------------------------------+
|OVERTAKE AND PASS INSUFFICIENT CLEARANCE|
|UNDER INFLUENCE - ALCOHOL               |
|WRONG SIDE - NOT PASSING                |
|TAKING MEDICATION (EXPLAIN IN NARRATIVE)|
|IMPROPER START FROM PARKED POSITION     |
|DISTRACTION IN VEHICLE                  |
|UNDER INFLUENCE - DRUG                  |
|FAILED TO YIELD ROW - TURN ON RED       |
|NA                                      |
|PARKED IN TRAFFIC LANE                  |
|ILL (EXPLAIN IN NARRATIVE)              |
|FAILED TO YIELD ROW - STOP SIGN         |
|UNSAFE SPEED                            |
|FAILED TO STOP AT PROPER PLACE          |
|FAILED TO YIELD ROW - PRIVATE DRIVE     |
|DISREGARD STOP SIGN OR LIGHT            |
|PASSED ON RIGHT SHOULDER                |
|DISREGARD TURN MARKS AT INTERSECTION    |
|FAILED TO PASS TO LEFT SAFELY           |
|LOAD NOT SECURED                        |
+----------

In [92]:
from pyspark.sql.functions import col

In [93]:
df_Units.filter(col("VEH_BODY_STYL_ID").contains("MOTORCYCLE")).count()

784

In [94]:
df_Units.filter(col("VEH_BODY_STYL_ID").contains("POLICE MOTORCYCLE")).count()

3

In [96]:
df_Units.filter(df_Units.VEH_BODY_STYL_ID == "MOTORCYCLE").count()

781

In [112]:
df_Units.printSchema()

root
 |-- CRASH_ID: string (nullable = true)
 |-- UNIT_NBR: string (nullable = true)
 |-- UNIT_DESC_ID: string (nullable = true)
 |-- VEH_PARKED_FL: string (nullable = true)
 |-- VEH_HNR_FL: string (nullable = true)
 |-- VEH_LIC_STATE_ID: string (nullable = true)
 |-- VIN: string (nullable = true)
 |-- VEH_MOD_YEAR: string (nullable = true)
 |-- VEH_COLOR_ID: string (nullable = true)
 |-- VEH_MAKE_ID: string (nullable = true)
 |-- VEH_MOD_ID: string (nullable = true)
 |-- VEH_BODY_STYL_ID: string (nullable = true)
 |-- EMER_RESPNDR_FL: string (nullable = true)
 |-- OWNR_ZIP: string (nullable = true)
 |-- FIN_RESP_PROOF_ID: string (nullable = true)
 |-- FIN_RESP_TYPE_ID: string (nullable = true)
 |-- VEH_DMAG_AREA_1_ID: string (nullable = true)
 |-- VEH_DMAG_SCL_1_ID: string (nullable = true)
 |-- FORCE_DIR_1_ID: string (nullable = true)
 |-- VEH_DMAG_AREA_2_ID: string (nullable = true)
 |-- VEH_DMAG_SCL_2_ID: string (nullable = true)
 |-- FORCE_DIR_2_ID: string (nullable = true)
 |-- V

In [110]:
df_Units = df_Units.withColumn('TOT_CASUALTIES_CNT', df_Units[35] + df_Units[36])

In [111]:
len(df_Units.columns)

38

In [115]:
df_Units.select("TOT_INJRY_CNT", "DEATH_CNT", "total_casualties").show(40)

+-------------+---------+----------------+
|TOT_INJRY_CNT|DEATH_CNT|total_casualties|
+-------------+---------+----------------+
|            0|        0|             0.0|
|            0|        0|             0.0|
|            0|        0|             0.0|
|            0|        0|             0.0|
|            0|        0|             0.0|
|            1|        0|             1.0|
|            1|        0|             1.0|
|            0|        0|             0.0|
|            0|        0|             0.0|
|            0|        0|             0.0|
|            0|        0|             0.0|
|            0|        0|             0.0|
|            0|        0|             0.0|
|            0|        0|             0.0|
|            0|        0|             0.0|
|            1|        0|             1.0|
|            0|        0|             0.0|
|            0|        0|             0.0|
|            0|        0|             0.0|
|            0|        0|             0.0|
|          

In [160]:
# df = df_Units.na.drop(subset=["VEH_MAKE_ID"])
df = df_Units.filter(df.VEH_MAKE_ID != "NA")

In [161]:
df = df.select("VEH_MAKE_ID", "total_casualties").groupby("VEH_MAKE_ID").sum("total_casualties").orderBy(col("sum(total_casualties)").desc())


In [162]:
df.select("VEH_MAKE_ID", "sum(total_casualties)").show()

+-------------+---------------------+
|  VEH_MAKE_ID|sum(total_casualties)|
+-------------+---------------------+
|    CHEVROLET|               7024.0|
|         FORD|               6992.0|
|       TOYOTA|               4228.0|
|        DODGE|               3146.0|
|       NISSAN|               3118.0|
|        HONDA|               2892.0|
|          GMC|               1256.0|
|      HYUNDAI|               1103.0|
|          KIA|               1049.0|
|         JEEP|                989.0|
|     CHRYSLER|                956.0|
|        MAZDA|                711.0|
|   VOLKSWAGEN|                582.0|
|      PONTIAC|                564.0|
|        LEXUS|                523.0|
|        BUICK|                521.0|
|   MITSUBISHI|                510.0|
|     CADILLAC|                498.0|
|      MERCURY|                397.0|
|MERCEDES-BENZ|                394.0|
+-------------+---------------------+
only showing top 20 rows



In [136]:
df_top15 = df.limit(15)
df_top5 = df.limit(5)
df_top_5_15 = df_top15.subtract(df_top5)

In [168]:
temp = df_top_5_15.select("VEH_MAKE_ID", "sum(total_casualties)").collect()

In [170]:
temp[0][0]

'HONDA'

In [167]:
df_Units.dropna(subset = ["VEH_PARKED_FL"]).select("VEH_PARKED_FL").show()

+-------------+
|VEH_PARKED_FL|
+-------------+
|            N|
|            N|
|            N|
|            N|
|            N|
|            N|
|            N|
|            N|
|            N|
|            N|
|            Y|
|            Y|
|            N|
|            N|
|            N|
|            N|
|            N|
|            Y|
|            Y|
|            Y|
+-------------+
only showing top 20 rows



In [290]:
df_damages.join(df_Units, on=["CRASH_ID"], how='inner').\
filter(((df_Units.VEH_DMAG_SCL_1_ID > "DAMAGED 4") & (~df_Units.VEH_DMAG_SCL_1_ID.isin(["NA", "NO DAMAGE", "INVALID VALUE"]))) | ((df_Units.VEH_DMAG_SCL_2_ID > "DAMAGED 4") & (~df_Units.VEH_DMAG_SCL_2_ID.isin(["NA", "NO DAMAGE", "INVALID VALUE"])))).\
filter(df_damages.DAMAGED_PROPERTY == "NONE").\
filter(col("FIN_RESP_TYPE_ID").contains("PROOF OF LIABILITY INSURANCE")).\
select("CRASH_ID", "VEH_DMAG_SCL_1_ID", "VEH_DMAG_SCL_2_ID", "FIN_RESP_TYPE_ID", "DAMAGED_PROPERTY").show(truncate=False)

+--------+-----------------+-----------------+----------------------------+----------------+
|CRASH_ID|VEH_DMAG_SCL_1_ID|VEH_DMAG_SCL_2_ID|FIN_RESP_TYPE_ID            |DAMAGED_PROPERTY|
+--------+-----------------+-----------------+----------------------------+----------------+
|14870169|DAMAGED 5        |NA               |PROOF OF LIABILITY INSURANCE|NONE            |
|14894076|DAMAGED 5        |NA               |PROOF OF LIABILITY INSURANCE|NONE            |
|14996273|DAMAGED 5        |NA               |PROOF OF LIABILITY INSURANCE|NONE            |
|15232090|DAMAGED 5        |NA               |PROOF OF LIABILITY INSURANCE|NONE            |
|15232090|DAMAGED 5        |DAMAGED 2        |PROOF OF LIABILITY INSURANCE|NONE            |
|15249931|DAMAGED 6        |NA               |PROOF OF LIABILITY INSURANCE|NONE            |
|15307513|DAMAGED 5        |NA               |PROOF OF LIABILITY INSURANCE|NONE            |
+--------+-----------------+-----------------+------------------------

In [ ]:
FIN_RESP_TYPE_ID

In [284]:
df_Units.select("VEH_DMAG_SCL_2_ID").distinct().show(truncate=False)

+-----------------+
|VEH_DMAG_SCL_2_ID|
+-----------------+
|DAMAGED 4        |
|INVALID VALUE    |
|NA               |
|DAMAGED 5        |
|DAMAGED 1 MINIMUM|
|DAMAGED 3        |
|NO DAMAGE        |
|DAMAGED 7 HIGHEST|
|DAMAGED 2        |
|DAMAGED 6        |
+-----------------+



In [268]:
df1 = df_Units.filter(~df_Units.VEH_DMAG_SCL_2_ID.isin(["NA", "NO DAMAGE", "INVALID VALUE"]) | ~df_Units.VEH_DMAG_SCL_1_ID.isin(["NA", "NO DAMAGE", "INVALID VALUE"])).\
filter(df_Units.VEH_DMAG_SCL_2_ID > "DAMAGED 4")

In [269]:
df2 = df_damages.filter(df_damages.DAMAGED_PROPERTY == "NONE")

In [270]:
df1.join(df2,  on=["CRASH_ID"], how='inner').count()

159